In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

In [2]:
path = os.getcwd() + "/atribuna/milarquivos"
content_file_array = []
classifier_array = []

In [3]:
def read_text(path, file_Name):
    with open(path, encoding="latin-1") as file:
        content_file = file.read()
        Build_DataFrame(content_file, file_Name)

In [4]:
def Build_DataFrame(content_file, file_Name):
    classifier =  GetClassifier(file_Name) 
    if(len(classifier) >= 3):        
        classifier_array.append(classifier)
        content_file_array.append(content_file)  

In [5]:
def GetClassifier(file_Name):
    name_split =  file_Name.split(".")
    first_name = name_split[0];
    return first_name[(len(first_name) - 3) : len(first_name)]

In [6]:
for file_Name in tqdm(os.listdir(path), desc="Carregar arquivos"):    
    if file_Name.endswith(".txt"):
        file_path = f"{path}\{file_Name}"
      
        # call read text file function
        read_text(file_path, file_Name)

Carregar arquivos: 100%|█████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 7567.91it/s]


In [7]:
textDataDrame =  pd.DataFrame({"Text":content_file_array, "Class":classifier_array})


In [8]:
textDataDrame.head(10)

,Text,Class
0,Nasce um novo amiguinho virtual\n<b> Coelhinho...,inf
1,Brigas na saída de boates\n<b> A denúncia é de...,poc
2,Queijo fica perto do livro dos recordes\n<b> O...,poc
3,Queijo fica perto do livro dos recordes\n<b> O...,reg
4,PLENÁRIO\n<b> </b><br>\n <br>\nBolo social do...,pot
5,Quadrilha assalta lanchonete\n<b> Cinco bandid...,poc
6,CARTAS\n<b> </b><br>\n <br>\nParabéns\nÀ Rede...,opi
7,Mineiro é baleado em assalto \n<b> O gerente c...,poc
8,CARTAS\n<b> </b><br>\n <br>\nParabéns\nVenho ...,opi
9,Aluno é morto com um soco\n<b> Adolescente de ...,poc


# StopWords removing

In [9]:
#!conda install unidecode

import nltk
import unidecode
from bs4 import BeautifulSoup

# nltk.download()
#- averaged_perceptron_tagger
# - floresta
# - mac_morpho
# - machado
# - punkt
# - stopwords
# - wordnet
# - words

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [10]:
def addStopWordsToList(texto):
    example_sent = texto
    word_tokens = word_tokenize(example_sent)

    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]

    filtered_sentence = ""

    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence += w + " "
    return filtered_sentence.strip()

In [13]:
stop_words = set(stopwords.words('portuguese'))
def returnDataFrameStopWords(dataFrame):
    dataReturn = pd.DataFrame()
    for row in tqdm(dataFrame.itertuples(index=True, name='Pandas'), desc="Limpeza de texto"):
        Text = row.Text
        Class = row.Class

        Text = BeautifulSoup(Text, "lxml").text
        Text = unidecode.unidecode(Text)
        Text = addStopWordsToList(Text)
        Text = Text.replace(" . "," ")
        Text = Text.replace(" , "," ")
        Text = Text.replace(" - "," ")
        Text = Text.replace(" : "," ")
        Text = Text.lower()

        linha = pd.Series([Text, Class])
        linha = pd.DataFrame([linha])
        dataReturn = pd.concat([linha, dataReturn], ignore_index=True)

    return dataReturn

In [14]:
df_text_clean = returnDataFrameStopWords(textDataDrame)

Limpeza de texto: 25it [00:00, 139.84it/s]


## Fatores de Ponderação

In [15]:
df_text_clean = df_text_clean.rename(columns = {0: 'Text', 1: 'Class'}, inplace = False)
df_text_clean.head()

,Text,Class
0,itapemirim ganha parque ecologico cachoeiro o ...,reg
1,qual a bronca ? perigo estrada o aposentado ca...,bro
2,rua pavimentacao o auxiliar servicos gerais va...,eco
3,aliado lula vai ser investigado roberto jeffer...,pot
4,cartas casa vovo duda pai viajando duas vamos ...,opi


In [16]:
from sklearn.feature_extraction.text import TfidfTransformer # tf-idf
from sklearn.feature_extraction.text import CountVectorizer # tf

In [17]:
classes =  ["at2","sro","inf","imo","ept","esp","fam","fam","tvt","cid","mul","pot","tav","mic","opi","poc","reg","eco","bro","cit","con","int"]

## TF

In [18]:
vectorizer = CountVectorizer()
TF = vectorizer.fit_transform(df_text_clean.Text)
features_names =  vectorizer.get_feature_names()

## TF-IDF

In [19]:
from sklearn.pipeline import Pipeline

In [20]:
pipe = Pipeline([('count', CountVectorizer(vocabulary=features_names)),
                 ('tfid', TfidfTransformer())]).fit(df_text_clean.Text)

In [25]:
lista_palavras = []
for row in tqdm(df_text_clean.itertuples(index=True, name='Pandas')):
    palavras = row.Text.split(" ")
    for palavra in palavras:
        if(not len(palavra) <= 2):
            if(palavra not in lista_palavras):
                lista_palavras.append(palavra)

25it [00:00, 155.53it/s]


# frequencia da palavra por documento

In [31]:
df_relacao_doc = pd.DataFrame()
df_relacao_doc['Index'] = lista_palavras

for i in tqdm(df_text_clean.index):
    doc = df_text_clean.iloc[i].Text
    
    lista_quant =[]
    
    for termo in lista_palavras:
        quant = doc.split(" ").count(termo)
        lista_quant.append(quant)
        
    df_relacao_doc[f'{i}'] = lista_quant
    
    
df_relacao_doc = df_relacao_doc.set_index('Index')

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:02<00:00, 11.83it/s]


In [117]:
df_text_clean.head()

,Text,Class
0,itapemirim ganha parque ecologico cachoeiro o ...,reg
1,qual a bronca ? perigo estrada o aposentado ca...,bro
2,rua pavimentacao o auxiliar servicos gerais va...,eco
3,aliado lula vai ser investigado roberto jeffer...,pot
4,cartas casa vovo duda pai viajando duas vamos ...,opi


In [36]:
df_relacao_doc.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Index,,,,,,,,,,,,,,,,,,,,,
itapemirim,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ganha,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
parque,5,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
ecologico,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cachoeiro,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# frequencia palavra por classe

In [108]:
dic_doc_class = {}

for classe in tqdm(classes):
    lista_doc = []
    for row in df_text_clean.itertuples(index=True, name='Pandas'):  
        if (str(classe) == row.Class):
            lista_doc.append(row.Text)
    dic_doc_class[f'{classe}'] = lista_doc

100%|████████████████████████████████████████████████████████████████████████████████| 22/22 [00:00<00:00, 1865.11it/s]


In [125]:
df_relacao_class = pd.DataFrame()
df_relacao_class['Index'] = lista_palavras
for key in tqdm(dic_doc_class):
    lista_docs =  dic_doc_class[key]
    docs = ""
    for doc in lista_docs:
        docs += f' {doc}'
        
    doc = docs
    
    lista_quant =[]
    
    for termo in lista_palavras:
        quant = doc.split(" ").count(termo)
        lista_quant.append(quant)
        
    df_relacao_class[f'{key}'] = lista_quant
    
    
df_relacao_class = df_relacao_class.set_index('Index')

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:02<00:00,  9.03it/s]


In [127]:
df_relacao_class.head()

,at2,sro,inf,imo,ept,esp,fam,tvt,cid,mul,...,tav,mic,opi,poc,reg,eco,bro,cit,con,int
Index,,,,,,,,,,,,,,,,,,,,,
itapemirim,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,5,0,0,0,0,0
ganha,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
parque,0,0,0,0,0,0,0,0,0,0,...,0,0,1,4,5,0,0,0,0,0
ecologico,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
cachoeiro,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0


## ICF

In [21]:
import math 

In [ ]:
for row in tqdm(df_relacao_class.itertuples(index=True, name='Pandas')):
    ci = 0
    for classe in classes:
        for i in row:
            print(i)
        if row!= 0:
            ci+=1
    break

In [22]:
def classesQuant():
    classes_quant = {}
    for classe in classes:
        quant = 0;
        for row in df_text_clean.itertuples(index=True, name='Pandas'):
            Class = row.Class
            if(classe == Class):
                quant+=1    
        classes_quant[classe] = quant
    return classes_quant

In [23]:
def calcICF(classes_quant):
    icfs = {}
    C = len(classes)
    for key in classes_quant:
        icfi = f'icf_{key}'
        Ci = classes_quant[key]
        valor = math.log((1 + (C/Ci)))
        icfs[icfi] = valor
    return icfs

In [24]:
def calculateICF():
    return calcICF(classesQuant())

In [ ]:
calculateICF()

## Machine Learning

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve,classification_report,r2_score
from sklearn import model_selection as ms
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [ ]:
le = LabelEncoder()

In [ ]:
def toNumeric(nome_coluna, coluna):
    df_text_clean[nome_coluna] = le.fit_transform (coluna)

In [ ]:
toNumeric('numeric_class',df_text_clean.Class)

In [ ]:
# features
X = TF

In [ ]:
#response
Y = df_text_clean['numeric_class'].copy()

In [ ]:
x_train, x_test, y_train, y_test = ms.train_test_split(X,Y,test_size=0.33, random_state=5)

In [ ]:
model_knn = KNeighborsClassifier(n_neighbors=1)

In [ ]:
model_knn.fit(x_train, y_train)

In [ ]:
pred = model_knn.predict_proba(x_test)

In [ ]:
plot_roc_curve(np.array(y_test), pred[:,1])

# Imports do EDA

In [ ]:
# print da curva ROC
def plot_roc_curve(y_true, y_score, figsize=(10,6)):
    fpr, tpr, _ = roc_curve(y_true, y_score, pos_label=2)
    plt.figure(figsize=figsize)
    auc_value = roc_auc_score(y_true, y_score)
    plt.plot(fpr, tpr, color='orange', label='ROC curve (area = %0.2f)' % auc_value)
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('Taxa de falso positivo')
    plt.ylabel('Taxa de verdadeiro positivo')
    plt.title('Curva ROC')
    plt.legend()
    lt.show()

In [ ]:
#precision    recall  f1-score   support
print(classification_report(y_test, pred, labels=[1, 2, 3]))

In [ ]:
# AUC
print('AUC: %0.2f' % roc_auc_score(y_test, pred))

## Número de amostras de cade classe


In [ ]:
import seaborn as sns

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.countplot(x = 'Class',data = textDataDrame)